# NLP con Long-Short Term Memory (LSTM)

##### Jean Pierre Londoño González
##### Mini-Proyecto de clasificación de texto con LSTM
##### 23AGO2025

En este notebook se implementa un clasificador de titulares de noticias en español utilizando una red neuronal LSTM. Las noticias fueron tomadas de la página de RCN y los titulares se clasificaron en cinco categorías: salud, tecnología, Colombia, economía y deportes. Para la tokenización se emplean las utilidades de la librería Hugging Face Transformers.

#### Referencias
- Dataset: https://huggingface.co/datasets/Nicky0007/titulos_noticias_rcn_clasificadas

### 1. Importación de librerias y carga de modelos

Inicio importando las librerías necesarias para el procesamiento de lenguaje natural, la manipulación de datos y la construcción del modelo. Esto incluye spaCy y NLTK para análisis lingüístico, stemming y léxicos como WordNet y SentiWordNet; pandas y NumPy para manejo de datos; y PyTorch junto con PyTorch Lightning para definir, entrenar y evaluar el modelo. Finalmente, se incluyen scikit-learn y torchmetrics para calcular métricas de rendimiento como precisión y reportes de clasificación.

In [81]:
import warnings  # Control y filtrado de mensajes de advertencia
# Desactiva advertencias para mantener limpia la salida
warnings.filterwarnings('ignore')

# Procesamiento de lenguaje natural y utilidades
import numpy as np  # Cálculo numérico y manejo de arreglos multidimensionales
import spacy  # Biblioteca para NLP avanzada (tokenización, POS tagging, dependencias, etc.)
import pandas as pd  # Manipulación y análisis de datos en estructuras tipo DataFrame

import nltk  # Biblioteca general de NLP (tokenización, stemming, stopwords, etc.)
from nltk.corpus import sentiwordnet as swn  # Léxico de sentimientos basado en WordNet
from nltk.corpus import wordnet as wn  # Diccionario semántico de palabras en inglés
from nltk.corpus import stopwords  # Lista de palabras vacías (stopwords) en múltiples idiomas
from nltk.stem.snowball import SpanishStemmer  # Algoritmo de stemming para español

import re  # Expresiones regulares para búsqueda y limpieza de texto
import unicodedata  # Normalización de caracteres Unicode (acentos, tildes, etc.)
from collections import Counter  # Conteo de frecuencias de elementos (tokens, palabras, etc.)

from datasets import load_dataset, concatenate_datasets # Carga y manipulación de datasets de Hugging Face
import os  # Manejo de rutas y operaciones del sistema de archivos

# Deep Learning con PyTorch
import torch  # Librería principal de tensores y operaciones en GPU
import torch.nn as nn  # Definición de capas de redes neuronales
import torch.nn.functional as F  # Funciones de activación y operaciones de redes
from torch.utils.data import Dataset  # Clase base para crear datasets personalizados
from torch.utils.data import random_split  # División aleatoria de datasets en train/test
from torch.utils.data import DataLoader  # Manejo eficiente de lotes (batches) de datos
from torch.utils.data import Subset

# Entrenamiento estructurado con PyTorch Lightning
from pytorch_lightning import LightningModule, Trainer  # Clase base y manejador de entrenamiento
from pytorch_lightning.loggers import TensorBoardLogger  # Registro de métricas en TensorBoard
from pytorch_lightning.callbacks.early_stopping import EarlyStopping  # Detener entrenamiento temprano si no mejora
from torchmetrics import Accuracy  # Métrica de precisión para clasificación

# Tipado para mayor legibilidad y validación
from typing import Tuple, Dict  # Definición de tipos de datos para funciones y estructuras

# Métricas de evaluación con Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  

import optuna

### 2. Cargue de datos

El dataset contiene titulares de noticias en español organizados por categorías. Está disponible en el Hugging Face Hub, lo que permite descargarlo fácilmente y utilizarlo directamente en entrenamientos o pruebas de modelos de NLP.

In [82]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
dataset = load_dataset("Nicky0007/titulos_noticias_rcn_clasificadas")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'url'],
        num_rows: 7224
    })
    test: Dataset({
        features: ['text', 'label', 'url'],
        num_rows: 1806
    })
})

El dataset incluye tres columnas: el título de la noticia (text), la categoría asociada (label) y el enlace a la publicación original (url). Para entrenar y validar el modelo, se dispone de 7,224 titulares en el conjunto de entrenamiento y 1,806 en el conjunto de prueba.

In [83]:
tr_dataset = dataset["train"]
te_dataset = dataset["test"]
full_dataset = concatenate_datasets([tr_dataset, te_dataset])

Observemos uno de sus registros

In [84]:
full_dataset[0]

{'text': 'Fútbol ¡Sonríe el fútbol! Médicos confirman que la salud de Pelé está mejorando Médicos que atienden al exjugador brasileño en Sao Paulo afirman que su condición de salud está mejorando. ',
 'label': 'deportes',
 'url': 'https://www.noticiasrcn.com/deportes/salud-de-pele-esta-mejorando-435134'}

In [85]:
pd.Series(full_dataset['label']).value_counts()

deportes      1806
tecnologia    1806
colombia      1806
salud         1806
economia      1806
Name: count, dtype: int64

Se valida que el dataset esta balanceado, pues cada categoria tiene 1806 registros.

Para los efectos de la tarea nos servirá el texto y la categoría.

A manera general, observemos que tan largos o cortos tienden a ser los textos.

In [86]:
text_lengths = [len(row['text']) for row in full_dataset]
print(f"Texto más corto: {min(text_lengths)}")
print(f"Texto más largo: {max(text_lengths)}")
print(f"Longitud promedio: {sum(text_lengths) / len(text_lengths)}")

Texto más corto: 63
Texto más largo: 362
Longitud promedio: 210.4736434108527


Estos valores son la cantidad de *caractéres* que tiene las secuencias. Una decisión ingenua pero útil en este momento podría ser ajustar la longitud de las secuencias que vamos a usar para el entrenamiento a unos 300 tokens. Esto podría ser suficiente para capturar una porción significativa de los textos.

### 3. Definición del Tokenizer

Ahora, vamos a definir el tokenizer para nuestra tarea. Para mantener las cosas simples, vamos a mantener un conteo de palabras y vamos a hacer un corte hasta los primeros 50mil tokens.

In [87]:
# Descargar recursos de NLTK
nltk.download("stopwords", quiet=True)

spanish_stopwords = set(stopwords.words("spanish"))
stemmer = SpanishStemmer()

def simple_tokenizer(text, remove_stopwords=True, stemming=True, normalize_accents=True):
    # 1. Minúsculas
    text = text.lower()
    
    # 2. Normalizar acentos
    if normalize_accents:
        text = ''.join(
            c for c in unicodedata.normalize('NFD', text)
            if unicodedata.category(c) != 'Mn'
        )
    
    # 3. Mantener solo letras (a-z, ñ)
    text = re.sub(r"[^a-zñ]+", " ", text)
    
    # 4. Tokenizar por espacios
    tokens = text.strip().split()
    
    # 5. Eliminar stopwords
    if remove_stopwords:
        tokens = [t for t in tokens if t not in spanish_stopwords]
    
    # 6. Stemming
    if stemming:
        tokens = [stemmer.stem(t) for t in tokens]
    
    return tokens

In [88]:
# Construimos el vocabulario a partir de conjunto de datos.
token_counts = Counter()
for text in full_dataset["text"]:
    token_counts.update(simple_tokenizer(text))

# 50k tokens
top_n_tokens = list(token_counts.keys())[:18640-2]
vocab = {"[PAD]": 0, "[UNK]": 1}
for token in top_n_tokens:
    vocab[token] = len(vocab)

def tokenize_text(text, max_length=50):
    tokens = simple_tokenizer(text)
    ids = [vocab.get(tok, vocab["[UNK]"]) for tok in tokens[:max_length]]
    ids += [vocab["[PAD]"]] * (max_length - len(ids))
    return ids

Exploremos ahora el tokenizador obtenido.

In [89]:
print(f"Vocabulario: {len(vocab)} tokens")
print("Primeros 15 tokens:")
print(f"{top_n_tokens[:15]}")
print("15 tokens de en medio:")
print(f"{top_n_tokens[1000:1015]}")
print("Últimos 15 tokens:")
print(f"{top_n_tokens[-15:]}")

Vocabulario: 9890 tokens
Primeros 15 tokens:
['futbol', 'sonri', 'medic', 'confirm', 'salud', 'pel', 'mejor', 'atiend', 'exjug', 'brasilen', 'sao', 'paul', 'afirm', 'condicion', 'especial']
15 tokens de en medio:
['competidor', 'pronost', 'crecimient', 'decret', 'desastr', 'ola', 'invernal', 'territori', 'viruel', 'mon', 'juli', 'institut', 'ins', 'dio', 'dias']
Últimos 15 tokens:
['oyent', 'acrobaci', 'nortemaerican', 'stress', 'comprometeri', 'samb', 'suramerican', 'resolu', 'circular', 'agorafobi', 'playback', 'fuel', 'albirroj', 'parpade', 'evoc']


Esta forma de exploración es para darnos una idea de las palabras más utilizadas en el corpus y nos dará un indicio de si la tokenización es adecuada o no. Vemos que tenemos algunos stop words, como artículos (el, la) y conectores (del, que). Para una tarea de clasificación de texto podríamos prescindir de estos pero para facilitar las cosas y ya que los demás tokens lucen bien, podemos preservarlos.

Lo que obtenemos son los ids de cada token según el vocabulario. Algo clave aquí es el padding: durante el entrenamiento necesito que todas las secuencias tengan la misma longitud para trabajar con matrices de forma cómoda. Como los textos no siempre tienen el mismo tamaño, si una secuencia es más larga la recorto, y si es más corta, la relleno con un token especial de padding. En este caso, definí que todas las cadenas tengan 8 tokens, por lo que las más cortas se completan hasta llegar a ese número.

### 4. Definición del dataset de pytorch

Ahora podemos proceder a definir el dataset.

In [90]:
class SpanishNewsDataset(Dataset):

    def __init__(self, tokenizer, dataset, seq_length: int = 512):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.seq_length = seq_length
        self.id_2_class_map = dict(enumerate(np.unique(dataset[:]['label'])))
        self.class_2_id_map = {v: k for k, v in self.id_2_class_map.items()}
        self.num_classes = len(self.id_2_class_map)
    
    def __getitem__(self, index) -> Dict[str, torch.Tensor]:
        text, y = self.dataset[index]['text'], self.dataset[index]['label']
        y = self.class_2_id_map[y]
        data = {'input_ids': torch.tensor(self.tokenizer(text, max_length=self.seq_length))}
        data['y'] = torch.tensor(y)
        return data

    def __len__(self):
        return len(self.dataset)

Ahora instanciaremos el dataset entero. Para este experimento, definiremos un tamaño máximo de secuencia de 2048 **tokens**. Que según nuestra intuición arriba, debería ser suficiente para la tarea.

In [91]:
max_len = 512
spanish_news_dataset = SpanishNewsDataset(tokenize_text, full_dataset, seq_length=max_len)
assert len(spanish_news_dataset) == len(full_dataset)

Y luego, procedemos a hacer el train-val-test split y crear los dataloaders.

In [92]:
# Extraer labels
labels = [int(spanish_news_dataset[i]["y"]) for i in range(len(spanish_news_dataset))]

# Indices de todo el dataset
indices = list(range(len(spanish_news_dataset)))

# Split estratificado: train (80%), temp (20%)
train_idx, temp_idx, _, temp_labels = train_test_split(
    indices, labels, test_size=0.2, stratify=labels, random_state=42
)

# Split estratificado: val (10%), test (10%) a partir del temp (20%)
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, stratify=temp_labels, random_state=42
)

# Construir Subsets
train_dataset = Subset(spanish_news_dataset, train_idx)
val_dataset   = Subset(spanish_news_dataset, val_idx)
test_dataset  = Subset(spanish_news_dataset, test_idx)

In [93]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

### 5. Definición del modelo LSTM

Ahora vamos a configurar un módulo pytorch simple para este problema. Vamos ha utilizar los embeddings, que vendrían siendo los vectores de palabra. Pytorch nos ofrece una capa con la que directamente podemos entrenarlos a partir de los token ids obtenidos. El resto consistirá en invocar una capa LSTM seguida de una capa densa para la clasificación.

Las redes recurrentes como las LSTM por diseño enlazan todas las dimensiones del vector de entrada, formando así la secuencia, la estructura natural que necesitamos representar.

In [94]:
class LSTMBlock(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
    
    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, _) = self.lstm(embedded)
        return hidden[-1]

### 6. Definición del clasificador

Finalmente, definimos el modelo en si. Este modelo constará de 3 capas:

- La tokenización, tal como la definimos anteriormente.
- El bloque LSTM, que acabamos de decinir.
- Una capa densa adicional que servirá como clasificador de aquello que nos entregue la capa del transformer.

Como este es un LightningModule, aquí definiremos el resto de funciones utilitarias para el entrenamiento de la tarea.

In [95]:
class SpanishNewsClassifierWithLSTM(LightningModule):

    def __init__(self, vocab_size: int, num_classes: int, emb_dim: int, hidden_dim: int = 128):
        super(SpanishNewsClassifierWithLSTM, self).__init__()
        self.num_classes = num_classes
        self.lstm = LSTMBlock(vocab_size, emb_dim, hidden_dim, num_classes)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
            nn.LogSoftmax(dim=1)
        )

        self.train_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.val_acc = Accuracy(task='multiclass', num_classes=num_classes)
        self.test_acc = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        embeddings = self.lstm(x)
        return self.classifier(embeddings)
    
    def training_step(self, batch, batch_idx):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.train_acc(y_hat, y)
        self.log('train-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train-acc', self.train_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.val_acc(y_hat, y)
        self.log('val-loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val-acc', self.val_acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def test_step(self, batch):
        x, y = batch['input_ids'], batch['y']
        y_hat = self(x)
        self.test_acc(y_hat, y)
        self.log('test-acc', self.test_acc, prog_bar=True, on_step=False, on_epoch=True)

    def predict_step(self, batch):
        x = batch['input_ids']
        return self(x)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=2)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val-loss"}

    
model = SpanishNewsClassifierWithLSTM(vocab_size=len(vocab) + 1, num_classes=spanish_news_dataset.num_classes, emb_dim=256)

tb_logger = TensorBoardLogger('tb_logs', name='LSTMClassifier')
callbacks=[EarlyStopping(monitor='val-loss', patience=3, mode='min')]
trainer = Trainer(max_epochs=10, devices=1, logger=tb_logger, callbacks=callbacks, precision=32)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | lstm       | LSTMBlock          | 2.9 M  | train
1 | classifier | Sequential         | 198 K  | train
2 | train_acc  | MulticlassAccuracy | 0      | train
3 | val_acc    | MulticlassAccuracy | 0      | train
4 | test_acc   | MulticlassAccuracy | 0      | train
----------------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.242    Total estimated model params size (MB)
16        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 452/452 [01:17<00:00,  5.83it/s, v_num=36]       
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 452/452 [01:17<00:00,  5.81it/s, v_num=36, val-loss=1.610, val-acc=0.199, train-loss=1.610, train-acc=0.200]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 452/452 [01:17<00:00,  5.82it/s, v_num=36, val-loss=1.610, val-acc=0.199, train-loss=1.610, train-acc=0.201]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 452/452 [01:17<00:00,  5.83it/s, v_num=36, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.194]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 452/452 [01:17<00:00,  5.81it/s, v_num=36, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.200]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 452/452 [01:17<00:00,  5.82it/s, v_num=36, val-loss=1.610, val-acc=0.199, train-loss=1.610, train-acc=0.197]
Validation: |      

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 452/452 [01:25<00:00,  5.29it/s, v_num=36, val-loss=1.610, val-acc=0.199, train-loss=1.610, train-acc=0.203]


In [96]:


def objective(trial):
    # Definir espacio de búsqueda
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)

    # Modelo con params del trial
    model = SpanishNewsClassifierWithLSTM(
        vocab_size=len(vocab)+1,
        num_classes=spanish_news_dataset.num_classes,
        emb_dim=128,
        hidden_dim=hidden_dim
    )
    model.classifier[3].p = dropout  # ajustar dropout dinámicamente

    trainer = Trainer(
        max_epochs=10,
        logger=TensorBoardLogger("optuna_logs", name=f"trial_{trial.number}"),
        enable_checkpointing=False
    )

    trainer.fit(model, train_loader, val_loader)
    # Retornar la métrica que quieres optimizar
    return trainer.callback_metrics["val-acc"].item()

# Ejecutar búsqueda
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best params:", study.best_params)

[I 2025-08-23 10:25:48,117] A new study created in memory with name: no-name-288af785-377f-49b0-888b-c28c5dfa6711
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | lstm       | LSTMBlock          | 1.3 M  | train
1 | classifier | Sequential         | 165 K  | train
2 | train_acc  | MulticlassAccuracy | 0      | train
3 | val_acc    | MulticlassAccuracy | 0      | train
4 | test_acc   | MulticlassAccuracy | 0      | train
----------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.060     Total estimated model params size (MB)
16        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 452/452 [00:48<00:00,  9.39it/s, v_num=3]        
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 452/452 [00:47<00:00,  9.50it/s, v_num=3, val-loss=1.610, val-acc=0.199, train-loss=1.610, train-acc=0.200]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 452/452 [00:46<00:00,  9.83it/s, v_num=3, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.199]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 452/452 [00:46<00:00,  9.64it/s, v_num=3, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.191]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 452/452 [00:47<00:00,  9.50it/s, v_num=3, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.192]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 452/452 [00:48<00:00,  9.39it/s, v_num=3, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.196]
Validation: |          |

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 452/452 [00:50<00:00,  8.91it/s, v_num=3, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.190]

[I 2025-08-23 10:34:10,869] Trial 0 finished with value: 0.20044296979904175 and parameters: {'hidden_dim': 64, 'dropout': 0.17765143036521747, 'lr': 0.0001462935125020138}. Best is trial 0 with value: 0.20044296979904175.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | lstm       | LSTMBlock          | 2.2 M  | train
1 | classifier | Sequential         | 264 K  | train
2 | train_acc  | MulticlassAccuracy | 0      | train
3 | val_acc    | MulticlassAccuracy | 0      | train
4 | test_acc   | MulticlassAccuracy | 0      | train
----------------------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.807     Total estimated model params size (MB)
16        Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 452/452 [02:31<00:00,  2.98it/s, v_num=1]        
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 452/452 [02:31<00:00,  2.99it/s, v_num=1, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.198]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 452/452 [02:29<00:00,  3.02it/s, v_num=1, val-loss=1.610, val-acc=0.199, train-loss=1.610, train-acc=0.188]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 452/452 [02:31<00:00,  2.99it/s, v_num=1, val-loss=1.610, val-acc=0.199, train-loss=1.610, train-acc=0.199]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 452/452 [02:30<00:00,  3.00it/s, v_num=1, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.198]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 452/452 [02:31<00:00,  2.97it/s, v_num=1, val-loss=1.610, val-acc=0.200, train-loss=1.610, train-acc=0.196]
Validation: |          |


Detected KeyboardInterrupt, attempting graceful shutdown ...
[W 2025-08-23 10:58:56,098] Trial 1 failed with parameters: {'hidden_dim': 256, 'dropout': 0.17875333164410417, 'lr': 0.0008516207363337893} because of the following error: NameError("name 'exit' is not defined").
Traceback (most recent call last):
  File "C:\Users\jepil\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\call.py", line 47, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "C:\Users\jepil\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 575, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "C:\Users\jepil\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 982, in _run
    results = self._run_stage()
  File "C:\Users\jepil\anaconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 1026, in _run_stage
    self.fit_loop.run()
  File "C:\Users\jepil\anaconda3\envs\nlp\lib\site-pac

NameError: name 'exit' is not defined

Observemos el proceso de entrenamiento

In [98]:
%load_ext tensorboard

In [99]:
%tensorboard --logdir tb_logs/

Reusing TensorBoard on port 6006 (pid 20796), started 1 day, 12:15:40 ago. (Use '!kill 20796' to kill it.)

### 7. Evaluación de desempeño del modelo

Realizamos la validación contra el conjunto de prueba.

In [100]:
model.eval()
trainer.test(model, test_loader)

Testing DataLoader 0: 100%|██████████| 57/57 [00:03<00:00, 15.30it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test-acc          │    0.20044296979904175    │
└───────────────────────────┴───────────────────────────┘

[{'test-acc': 0.20044296979904175}]

### 8. Resultados

Finalmente, vamos a hacer uso del modelo y ver que tan bueno es para la clasificación de titulares de noticias.

In [101]:
predictions = trainer.predict(model, test_loader)
predictions = torch.cat(predictions, dim=0)
predictions = torch.argmax(predictions, dim=-1)
predictions = [spanish_news_dataset.id_2_class_map[pred] for pred in predictions.numpy()]

Predicting DataLoader 0: 100%|██████████| 57/57 [00:03<00:00, 15.66it/s]


In [102]:
test_indices = test_dataset.indices
df = pd.DataFrame(data={
    "texto": full_dataset[test_indices]['text'],
    "tokens": [tokenize_text(v) for v in full_dataset[test_indices]['text']],
    "categoría": full_dataset[test_indices]['label'],
    'predicción': predictions
}, index=test_indices)

id_2_token = {v: k for k, v in vocab.items()}

df['tokens_string'] = df.tokens.apply(lambda t: ' '.join([id_2_token[i] for i in t]))
df = df[["texto", "tokens", "tokens_string", "categoría", "predicción"]]
df.style.set_table_styles(
    [
        {'selector': 'td', 'props': [('word-wrap', 'break-word')]}
    ]
)
df.head(15)

,texto,tokens,tokens_string,categoría,predicción
6281,Declaración de renta Se viene la declaración d...,"[686, 1559, 3275, 686, 1559, 570, 7270, 92, 27...",declar rent vien declar rent sancion expon si ...,economia,colombia
8629,Vacunas covid Vacunas anticovid: lo que debe s...,"[270, 428, 270, 274, 93, 492, 412, 408, 172, 6...",vacun cov vacun anticov deb sab dosis refuerz ...,salud,colombia
5696,Gasolina ¿Más inflación? ABC de la relación en...,"[1554, 69, 990, 6146, 263, 467, 1554, 989, 496...",gasolin mas inflacion abc relacion aument gaso...,economia,colombia
1161,Vehículos Siguen en aumento: matrículas nuevas...,"[60, 275, 467, 456, 26, 1919, 1782, 1759, 409,...",vehicul sig aument matricul nuev automovil cre...,economia,colombia
3112,Catar 2022 Marruecos vs. Croacia: reviva los m...,"[386, 893, 33, 2077, 1608, 8, 215, 451, 275, 4...",cat marruec vs croaci reviv mejor moment part ...,deportes,colombia
5158,Congreso de la República ¿No se han graduado? ...,"[763, 764, 3362, 66, 5498, 763, 162, 104, 469,...",congres republ gradu nivel academ congres estu...,colombia,colombia
175,Empleo La petición de Mintrabajo para generar ...,"[784, 1517, 1518, 720, 812, 775, 383, 288, 123...",emple peticion mintrabaj gener cambi economi p...,economia,colombia
1353,Salud mental Así ayuda la música a mejorar la ...,"[6, 2060, 284, 512, 3353, 8, 6, 2060, 4242, 23...",salud mental asi ayud music mejor salud mental...,salud,colombia
1130,Crisis en Ucrania Exportación de granos se vol...,"[2181, 131, 3119, 511, 1596, 3208, 3363, 136, ...",crisis ucrani export gran volvi impos bloque r...,economia,colombia
3465,Ministerio de Salud Colombia firmó acuerdo par...,"[172, 6, 18, 569, 373, 1263, 270, 2040, 2214, ...",ministeri salud colombi firm acuerd fabric vac...,salud,colombia


In [ ]:
errors = df[df['categoría'] != df['predicción']]
errors.head(15)

In [ ]:
df['predicción'].unique()

### 9. Conclusiones

#### Eficacia del flujo de análisis

- Se implementó un pipeline de clasificación multiclase sobre títulos de noticias en español utilizando un modelo basado en LSTM.
- El bloque LSTM actuó como featurizer, extrayendo representaciones de las secuencias de entrada a partir de las cuales se realizaron las predicciones.

#### Rendimiento del modelo

- El modelo alcanzó un accuracy de 0.2, lo que indica un desempeño bajo para la tarea de clasificación.
- Este resultado refleja que, en su configuración actual y sin optimización de hiperparámetros, el modelo no logra capturar de manera suficiente las características del texto para diferenciar entre las clases.

#### Limitaciones observadas

- El tiempo de entrenamiento fue elevado, lo cual es consistente con la naturaleza secuencial de las LSTM, donde en cada paso temporal se deben calcular gradientes.
- El modelo se entrenó sin realizar ajustes de hiperparámetros (tasa de aprendizaje, tamaño de batch, número de capas, etc.), lo cual limita su potencial de rendimiento.
- La representación basada únicamente en tokens y padding puede ser insuficiente para capturar matices semánticos complejos.

#### Áreas de mejora

- Explorar técnicas de optimización de hiperparámetros y regularización para mejorar la capacidad predictiva.
- Incorporar embeddings preentrenados en español (por ejemplo, FastText o Word2Vec) para enriquecer la representación semántica.
- Evaluar arquitecturas más modernas como GRU, Transformers o modelos preentrenados que podrían ofrecer un mejor trade-off entre rendimiento y tiempo de cómputo.

#### Valor práctico

- Aunque el rendimiento inicial es bajo, el experimento permite validar el pipeline y sentar bases para futuros ajustes.
- Este método ofrece un punto de partida funcional para experimentar con arquitecturas más sofisticadas y con un mejor ajuste de hiperparámetros, lo cual podría elevar significativamente la precisión del clasificador.